# MVP Pipeline de Dados
## Pesquisa sobre aparelhos celulares

Edmilson Prata da Silva

PUC-RJ - MBA em Ciência de Dados e Analytics

Disciplina de Engenharia de Dados

## Script ETL para carga na camada SILVER

## Imports

Imports das bibliotecas necessárias para o funcionamento do script.

In [0]:
import re
import uuid
import warnings
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

## Carga de Dados

Os dados serão carregados a partir da camada bronze para os tratamentos necessários.

In [0]:
spark = SparkSession.builder.getOrCreate()
df_spark = spark.table("bronze.mobile_devices")
df_spark.printSchema()


root
 |-- company_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- mobile_weight: string (nullable = true)
 |-- ram: string (nullable = true)
 |-- front_camera: string (nullable = true)
 |-- back_camera: string (nullable = true)
 |-- processor: string (nullable = true)
 |-- battery_capacity: string (nullable = true)
 |-- screen_size: string (nullable = true)
 |-- launched_price_pakistan: string (nullable = true)
 |-- launched_price_india: string (nullable = true)
 |-- launched_price_china: string (nullable = true)
 |-- launched_price_usa: string (nullable = true)
 |-- launched_price_dubai: string (nullable = true)
 |-- launched_year: string (nullable = true)



### Análise dos dados

Aqui será feita análise dos dados antes de iniciar o tratamento.

#### Conversão para Pandas

O dataset será convertido para DataFrame Pandas, devido ser baixo o volume de dados, para facilitar a análise.

In [0]:
# Utilizando Apache Arrow para otimizar o uso do Pandas
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Convertendo de Spark para Pandas
df_pd = df_spark.toPandas()

df_pd.shape

Out[123]: (930, 15)

#### Análise dos Dados

Avaliação do conteúdo do dataset para verificar a qualidade dos dados e os tratamentos necessários.

##### Dados nulos e faltantes

Exibindo informações sobre as colunas para verificar nulos e faltantes.

In [0]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   company_name             930 non-null    object
 1   model_name               930 non-null    object
 2   mobile_weight            930 non-null    object
 3   ram                      930 non-null    object
 4   front_camera             930 non-null    object
 5   back_camera              930 non-null    object
 6   processor                930 non-null    object
 7   battery_capacity         930 non-null    object
 8   screen_size              930 non-null    object
 9   launched_price_pakistan  930 non-null    object
 10  launched_price_india     930 non-null    object
 11  launched_price_china     930 non-null    object
 12  launched_price_usa       930 non-null    object
 13  launched_price_dubai     930 non-null    object
 14  launched_year            930 non-null    o

Já se pode observar, acima, que não há dados faltantes nem nulos para tratar no conjunto de dados.

##### Observação direta

Observação dos dados tais como constam no dataframe para identificação de problemas de forma visual.

In [0]:
df_pd.head()

,company_name,model_name,mobile_weight,ram,front_camera,back_camera,processor,battery_capacity,screen_size,launched_price_pakistan,launched_price_india,launched_price_china,launched_price_usa,launched_price_dubai,launched_year
0,Apple,iPhone 16 128GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 224,999","INR 79,999","CNY 5,799",USD 799,"AED 2,799",2024
1,Apple,iPhone 16 256GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 234,999","INR 84,999","CNY 6,099",USD 849,"AED 2,999",2024
2,Apple,iPhone 16 512GB,174g,6GB,12MP,48MP,A17 Bionic,"3,600mAh",6.1 inches,"PKR 244,999","INR 89,999","CNY 6,499",USD 899,"AED 3,199",2024
3,Apple,iPhone 16 Plus 128GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,"PKR 249,999","INR 89,999","CNY 6,199",USD 899,"AED 3,199",2024
4,Apple,iPhone 16 Plus 256GB,203g,6GB,12MP,48MP,A17 Bionic,"4,200mAh",6.7 inches,"PKR 259,999","INR 94,999","CNY 6,499",USD 949,"AED 3,399",2024


Observa acima que o conjunto de dados trás, em todas as colunas, a unidade de medida e o símbolo de moeda. É preciso separar o valor do dado de sua unidade de medida.

##### Verificação de duplicidades

O campo de maior interesse, que identifica unicamente o aparelho de celular, é o nome do modelo. Vamos verificar se há duplicidades. Porém, antes, é necessário eliminar linhar inteiras que estejam duplicadas, pois estas não necessitam verificações mais sofisticadas.

In [0]:
print(f"Há duplicidades? {df_pd.duplicated().any()}")
df_pd.drop_duplicates(inplace=True)
print(f"Redução de {df_spark.count()} para {df_pd.shape[0]} linhas após remoção de duplicatas.")
print(f"Valores únicos na coluna model_name: {df_pd['model_name'].unique().size}")

Há duplicidades? True
Redução de 930 para 915 linhas após remoção de duplicatas.
Valores únicos na coluna model_name: 908


Observase que a quantidade de valores únicos é menor que a quantidade total de linhas do conjunto. Isso comprova a duplicidade. A seguir, será analisado se a duplicidade pode ser solucionada com características que possam identificar o modelo de forma única.

In [0]:
df_pd.reset_index()
duplicados = df_pd[df_pd['model_name'].duplicated(keep=False)]
duplicados.sort_values(by='model_name', inplace=True)
duplicados

,company_name,model_name,mobile_weight,ram,front_camera,back_camera,processor,battery_capacity,screen_size,launched_price_pakistan,launched_price_india,launched_price_china,launched_price_usa,launched_price_dubai,launched_year
476,Realme,P2 Pro 5G 256GB,188g,12GB,32MP,50MP + 8MP + 50MP,Qualcomm Snapdragon 7+ Gen 3,"5,500mAh",6.78 inches,"PKR 89,999","INR 32,999","CNY 2,500",USD 340,"AED 1,210",2024
507,Realme,P2 Pro 5G 256GB,195g,12GB,16MP,50MP + 8MP,MediaTek Dimensity 9200,"5,100mAh",6.7 inches,"PKR 99,999","INR 38,999","CNY 2,900",USD 400,"AED 1,400",2024
315,Vivo,Pad 128GB,498g,6GB,8MP,13MP,Snapdragon 870,8040mAh,11 inches,"PKR 64,999","INR 34,999","CNY 2,499",USD 329,"AED 1,199",2022
321,iQOO,Pad 128GB,520g,6GB,8MP,13MP,Snapdragon 870,8040mAh,11 inches,"PKR 69,999","INR 37,999","CNY 2,699",USD 349,"AED 1,299",2023
509,Realme,Pad 128GB,440g,6GB,8MP,8MP,MediaTek Helio G80,"7,100mAh",10.4 inches,"PKR 64,999","INR 22,999","CNY 1,600",USD 230,AED 850,2021
316,Vivo,Pad 2 256GB,535g,8GB,8MP,50MP,MediaTek Dimensity 1200,8040mAh,11 inches,"PKR 74,999","INR 39,999","CNY 3,199",USD 399,"AED 1,399",2023
322,iQOO,Pad 2 256GB,540g,8GB,8MP,50MP,Snapdragon 888,8040mAh,12.3 inches,"PKR 79,999","INR 43,999","CNY 3,499",USD 399,"AED 1,499",2024
511,Realme,Pad 2 256GB,482g,8GB,8MP,8MP,MediaTek Helio G99,"8,360mAh",11.5 inches,"PKR 84,999","INR 31,999","CNY 2,300",USD 320,"AED 1,200",2023
318,Vivo,Pad 3 128GB,500g,6GB,8MP,13MP,Snapdragon 750G,8040mAh,11 inches,"PKR 62,999","INR 34,999","CNY 2,399",USD 349,"AED 1,199",2023
358,Oppo,Pad 3 128GB,533g,8GB,8MP,13MP,Dimensity 8350,9510mAh,11.6 inches,"PKR 99,999","INR 54,999","CNY 4,499",USD 699,"AED 2,499",2024


Exceto pelo primeiro modelo da lista, o "P2 Pro 5G 256GB", todos os demais são casos de modelos com nomes muito genéricos que acabaram coincidindo com o de algum concorrente. Podemos acrescentar o nome do fabricante ao modelo e, assim, resolver estes casos. Quanto ao modelo da "Realme", podemos resolver pela resolução da câmera frontal.

### Tratamento dos Dados

Serão feitos os tratamentos observados como necessários na análise. A qualidade geral dos dados está boa, restando separar os valores e as unidades de medidas, além de resolver as duplicidades nos nomes dos modelos.

#### Tratamento de duplicidades

Como os modelos aparentemente duplicados, na verdade, são coincidências de nomes com concorrentes ou de modelos com características físicas distintas, será utilizada como solução a criação de um identificador único, baseado em UUID, que posteriormente servirá como chave primária da tabelas de modelos.

In [0]:
df_pd['model_id'] = [str(uuid.uuid4()) for x in range(df_pd.shape[0])]
df_pd['model_id'].unique().size

Out[128]: 915

#### Tratamento de dados técnicos

Os dados que possuem unidades de medida misturadas com o valor do dado em si, e outras impurezas, serão higienizados e transformados em dados numéricos.

In [0]:
# mobile_weight: Removendo o "g" de grama e convertendo para inteiro.
df_pd['mobile_weight'] = df_pd['mobile_weight'].apply(lambda x: int(float(x.replace('g', ''))))

# front_camera: Removendo o "MP" de megapixel e convertendo para inteiro:
df_pd['front_camera'] = df_pd['front_camera'].apply(lambda x: float(re.sub(r"[^0-9\.]", "", x[0:x.find('MP')])))

# back_camera: Removendo o "MP" de megapixel e convertendo para inteiro:
df_pd['back_camera'] = df_pd['back_camera'].apply(lambda x: float(re.sub(r"[^0-9\.]", "", x[0:x.find('MP')])))

# screen_size: Removendo o "inches" de polegada e convertendo para float:
df_pd['screen_size'] = df_pd['screen_size'].apply(lambda x: float(re.sub(r"[^0-9\.]", "", x[0:x.find('inches')])))

# ram: Removendo o "GB" de gigabytes e convertendo para inteiro.
df_pd['ram'] = df_pd['ram'].apply(lambda x: int(float(re.sub(r"[^0-9]", "", x))))

# battery_capacity: Removendo o "mAh" de micro ampére e convertendo para inteiro:
df_pd['battery_capacity'] = df_pd['battery_capacity'].apply(lambda x: int(float(re.sub(r"[^0-9]", "", x))))

##### Tratamento de dados de preços

Tratamento de falta de padrão nos dados, tais como uso de "," e "." para decimais e milhares, símbolos de moeda e outros tratamentos são realizados.

In [0]:
def convert_price(price_str):
   
    price_str = '0' + re.sub(r"[^0-9\.,]", '', price_str).strip()
    
    # Caso 1: Vírgula como decimal (ex: "1.234,56" ou "1234,56")
    if ',' in price_str and (price_str.count('.') <= 1 or '.' not in price_str):
        return float(price_str.replace('.', '').replace(',', '.'))
    
    # Caso 2: Ponto como decimal (ex: "1,234.56" ou "1234.56")
    elif '.' in price_str:
        return float(price_str.replace(',', ''))
    
    # Caso 3: Sem separador decimal explícito (ex: "1234")
    else:
        return float(price_str.replace(',', '').replace('.', ''))

In [0]:
# launched_price_pakistan: Removendo o "PKR", símbolo da moeda do Paquistão, e convertendo para float:
df_pd['launched_price_pakistan'] = df_pd['launched_price_pakistan'].apply(lambda x: convert_price(x))

# launched_price_india: Removendo preços da Índia devido haver muitos erros:
df_pd.drop(columns=['launched_price_india'], inplace=True)

# launched_price_china: Removendo o "CNY", símbolo da moeda do China, e convertendo para float:
df_pd['launched_price_china'] = df_pd['launched_price_china'].apply(lambda x: convert_price(x))

# launched_price_usa: Removendo o "USD", símbolo da moeda dos USA, e convertendo para float:
df_pd['launched_price_usa'] = df_pd['launched_price_usa'].apply(lambda x: convert_price(x))

# launched_price_dubai: Removendo o "AED", símbolo da moeda de Dubai, e convertendo para float:
df_pd['launched_price_dubai'] = df_pd['launched_price_dubai'].apply(lambda x: convert_price(x))

##### Verificação de estatística sumarizada

Verificação para identificar problemas a partir da estátistica sumarizada, tais como conversões de valores que potencialmente falharam.

In [0]:
df_pd.describe()

,mobile_weight,ram,front_camera,back_camera,battery_capacity,screen_size,launched_price_pakistan,launched_price_china,launched_price_usa,launched_price_dubai
count,915.000000,915.000000,915.000000,915.000000,915.000000,915.000000,915.000000,915.000000,915.000000,915.000000
mean,228.910383,7.818579,18.166776,46.491585,5030.280874,7.092678,126.420894,55.876281,387.311970,188.170957
std,106.164722,3.195642,12.043474,31.124175,1365.571834,1.544578,102.059571,212.145618,289.767579,332.295706
min,135.000000,1.000000,2.000000,5.000000,2000.000000,5.000000,0.000000,1.050000,1.049000,1.000000
25%,185.000000,6.000000,8.000000,13.000000,4400.000000,6.500000,54.999000,1.999000,180.000000,1.699000
50%,195.000000,8.000000,16.000000,50.000000,5000.000000,6.670000,89.999000,3.299000,329.000000,3.199000
75%,209.000000,8.000000,32.000000,50.000000,5100.000000,6.780000,179.999000,6.299000,574.000000,10.799000
max,732.000000,16.000000,60.000000,200.000000,11200.000000,14.600000,604.999000,999.000000,1719.000000,999.000000


### Separação dos dados em tabelas

Organização dos dados de acordo com cada tabela que será armazenada na camada silver.

#### Tabela de empresas

Tabelas dimensão com dados das empresas concorrentes que são fabricantes de aparelhos celulares (silver.company).

In [0]:
company_df = pd.DataFrame()
company_df['company_name'] = df_pd['company_name'].unique()
company_df['company_id'] = [str(uuid.uuid4()) for x in range(company_df.shape[0])]
company_df

,company_name,company_id
0,Apple,b584fcf4-6fea-46d7-8a5b-2cc994df30f3
1,Samsung,05f2afcc-51bd-412e-87dd-b32ac7e2ff4e
2,OnePlus,c482a035-36fe-4b4a-8a6f-a3b10aa45304
3,Vivo,a2d25021-3c44-4554-886f-31d39c9ce5cf
4,iQOO,55db049c-f1c9-4760-b4f3-365a1310dc83
5,Oppo,67a5680f-b05e-43bc-bb18-c4e37156c89a
6,Realme,0e37c57d-fa83-4e60-bf8e-f58663a7a9d5
7,Xiaomi,fca2bc57-3f4e-4fce-860b-68c09e167af0
8,Lenovo,5f74fb21-5da8-4ca6-b857-8f5e26a1d891
9,Motorola,c5316e43-ffcc-4e48-afc1-1f1c1f16f00c


#### Tabela de modelos

Tabela dimensão com dados dos modelos de celulares e suas características técnicas (silver.model).

In [0]:
model_df = pd.DataFrame()
model_df = df_pd[[
  'model_id','model_name','mobile_weight',
  'ram','front_camera','back_camera',
  'processor','battery_capacity',
  'screen_size','launched_year'
]]
model_df.shape

Out[134]: (915, 10)

#### Tabela de preços

Tabela dimensão com dados dos preços dos modelos de celulares em cada país.

In [0]:
# Suprime o aviso SettingWithCopyWarning
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

price_df = pd.DataFrame()
temp_df = pd.DataFrame()

temp_df = df_pd[['model_id', 'launched_price_pakistan']]
temp_df = temp_df.rename(columns={'launched_price_pakistan': 'launched_price'})
temp_df['country'] = 'pakistan'
price_df = pd.concat([price_df, temp_df])

temp_df = df_pd[['model_id', 'launched_price_china']]
temp_df = temp_df.rename(columns={'launched_price_china': 'launched_price'})
temp_df['country'] = 'china'
price_df = pd.concat([price_df, temp_df])

temp_df = df_pd[['model_id', 'launched_price_usa']]
temp_df = temp_df.rename(columns={'launched_price_usa': 'launched_price'})
temp_df['country'] = 'usa'
price_df = pd.concat([price_df, temp_df])

temp_df = df_pd[['model_id', 'launched_price_dubai']]
temp_df = temp_df.rename(columns={'launched_price_dubai': 'launched_price'})
temp_df['country'] = 'dubai'
price_df = pd.concat([price_df, temp_df])

price_df.shape

Out[135]: (3660, 3)

#### Tabela de smartphones

Tabela com dados dos smartphones, que é a tabela fato do modelo (silver.smartphones).

In [0]:
company_dict = company_df.set_index('company_name')['company_id'].to_dict()
df_pd['company_id'] = df_pd['company_name'].apply(lambda x: company_dict.get(x))

smartphones_df = df_pd[['company_id', 'model_id']]
smartphones_df['smartphone_id'] = [str(uuid.uuid4()) for x in range(smartphones_df.shape[0])]
smartphones_df.shape

Out[136]: (915, 3)

### Persistência dos dados nas tabelas

A seguir, os dados da camada silver serão persistidos em suas respectivas tabelas.

In [0]:
# Grava os dados na tabela Delta:
def save_data(table_name, df_spark):
    df_spark \
    .write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(table_name)
    print(f"Dados carregados com sucesso na tabela {table_name}.")

#### Tabela silver.company

Persistência dos dados das empresas concorrentes.

In [0]:
df_spark = spark.createDataFrame(company_df)
save_data('silver.company', df_spark)

Dados carregados com sucesso na tabela silver.company.


#### Tabela silver.model

Persistência dos dados dos modelos de smartphones.

In [0]:
df_spark = spark.createDataFrame(model_df)
df_spark = df_spark.withColumn("mobile_weight", col("mobile_weight").cast("decimal(10,2)"))
df_spark = df_spark.withColumn("front_camera", col("front_camera").cast("decimal(10,2)"))
df_spark = df_spark.withColumn("back_camera", col("back_camera").cast("decimal(10,2)"))
df_spark = df_spark.withColumn("screen_size", col("screen_size").cast("decimal(4,2)"))
df_spark = df_spark.withColumn("battery_capacity", col("battery_capacity").cast("integer"))
df_spark = df_spark.withColumn("launched_year", col("launched_year").cast("integer"))
df_spark = df_spark.withColumn("ram", col("ram").cast("integer"))
save_data('silver.model', df_spark)

Dados carregados com sucesso na tabela silver.model.


#### Tabela silver.prices

Persistência dos dados dos preços de lançamento dos modelos de smartphones.

In [0]:
df_spark = spark.createDataFrame(price_df)
df_spark = df_spark.withColumn("launched_price", col("launched_price").cast("decimal(10,2)"))
save_data('silver.price', df_spark)

Dados carregados com sucesso na tabela silver.price.


#### Tabela silver.smartphones

Persistência dos dados dos smartphones, tabela fato com chaves para as demais tabelas dimensão.

In [0]:
df_spark = spark.createDataFrame(smartphones_df)
save_data('silver.smartphones', df_spark)

Dados carregados com sucesso na tabela silver.smartphones.


### Teste de carga da Tabela

Teste de carga da tabela, para garantir o sucesso da operação.

In [0]:
%sql select * from silver.company limit 10

company_id,company_name
b584fcf4-6fea-46d7-8a5b-2cc994df30f3,Apple
05f2afcc-51bd-412e-87dd-b32ac7e2ff4e,Samsung
c482a035-36fe-4b4a-8a6f-a3b10aa45304,OnePlus
a2d25021-3c44-4554-886f-31d39c9ce5cf,Vivo
55db049c-f1c9-4760-b4f3-365a1310dc83,iQOO
67a5680f-b05e-43bc-bb18-c4e37156c89a,Oppo
0e37c57d-fa83-4e60-bf8e-f58663a7a9d5,Realme
fca2bc57-3f4e-4fce-860b-68c09e167af0,Xiaomi
5f74fb21-5da8-4ca6-b857-8f5e26a1d891,Lenovo
c5316e43-ffcc-4e48-afc1-1f1c1f16f00c,Motorola


In [0]:
%sql select * from silver.model limit 10

model_id,model_name,mobile_weight,ram,qt_total_cameras,front_camera,back_camera,processor,battery_capacity,screen_size,launched_year
dd913374-b682-49fc-bdc0-60a17a1278b7,iPhone 16 128GB,174.00,6,null,12.00,48.00,A17 Bionic,3600,6.10,2024
77b45527-e18d-45f7-a7dc-36c861bbddbe,iPhone 16 256GB,174.00,6,null,12.00,48.00,A17 Bionic,3600,6.10,2024
25ba81b5-2569-4f02-ad50-434da407d594,iPhone 16 512GB,174.00,6,null,12.00,48.00,A17 Bionic,3600,6.10,2024
1e206a76-5fe9-4c5e-952f-f44749cbe495,iPhone 16 Plus 128GB,203.00,6,null,12.00,48.00,A17 Bionic,4200,6.70,2024
848b43e9-70bc-4039-afd1-2ecd2a231d69,iPhone 16 Plus 256GB,203.00,6,null,12.00,48.00,A17 Bionic,4200,6.70,2024
87b31c5a-49e4-41c3-b508-6c72f13d100c,iPhone 16 Plus 512GB,203.00,6,null,12.00,48.00,A17 Bionic,4200,6.70,2024
ac5859c1-7502-4eaa-be85-2e7514c81d29,iPhone 16 Pro 128GB,206.00,6,null,12.00,50.00,A17 Pro,4400,6.10,2024
d950d4f3-ad3e-455a-a610-6e19a3a42a2a,iPhone 16 Pro 256GB,206.00,8,null,12.00,50.00,A17 Pro,4400,6.10,2024
5458c7b2-c0d3-4449-be3b-b12dc2912bd4,iPhone 16 Pro 512GB,206.00,8,null,12.00,50.00,A17 Pro,4400,6.10,2024
4809239c-fb16-41e8-ae07-8e4975eb9a1a,iPhone 16 Pro Max 128GB,221.00,6,null,12.00,48.00,A17 Pro,4500,6.70,2024


In [0]:
%sql select * from silver.price limit 10

price_id,model_id,country,launched_price
null,dd913374-b682-49fc-bdc0-60a17a1278b7,pakistan,225.00
null,77b45527-e18d-45f7-a7dc-36c861bbddbe,pakistan,235.00
null,25ba81b5-2569-4f02-ad50-434da407d594,pakistan,245.00
null,1e206a76-5fe9-4c5e-952f-f44749cbe495,pakistan,250.00
null,848b43e9-70bc-4039-afd1-2ecd2a231d69,pakistan,260.00
null,87b31c5a-49e4-41c3-b508-6c72f13d100c,pakistan,275.00
null,ac5859c1-7502-4eaa-be85-2e7514c81d29,pakistan,285.00
null,d950d4f3-ad3e-455a-a610-6e19a3a42a2a,pakistan,295.00
null,5458c7b2-c0d3-4449-be3b-b12dc2912bd4,pakistan,315.00
null,4809239c-fb16-41e8-ae07-8e4975eb9a1a,pakistan,315.00


In [0]:
%sql select * from silver.smartphones limit 10

smartphone_id,company_id,model_id
e3d1547a-ef06-4e14-8adf-8b3f40717d51,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,dd913374-b682-49fc-bdc0-60a17a1278b7
181da901-9cab-4215-bb70-19c1a110358a,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,77b45527-e18d-45f7-a7dc-36c861bbddbe
c7c8f20c-8768-4770-8571-7115e44d7059,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,25ba81b5-2569-4f02-ad50-434da407d594
5d65d589-fce5-4700-be2e-7d1d478031b0,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,1e206a76-5fe9-4c5e-952f-f44749cbe495
12394a52-55ef-470d-b331-15def6c44fad,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,848b43e9-70bc-4039-afd1-2ecd2a231d69
3da84573-799f-4b50-b1a0-5fcf5e944ac1,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,87b31c5a-49e4-41c3-b508-6c72f13d100c
594d47b8-d028-4ecd-ac65-d867123218e3,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,ac5859c1-7502-4eaa-be85-2e7514c81d29
baf77cdd-a6fc-4c1b-a4d7-7f825f37c39e,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,d950d4f3-ad3e-455a-a610-6e19a3a42a2a
eb739bd0-93b3-4573-9f17-0acac17ce57b,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,5458c7b2-c0d3-4449-be3b-b12dc2912bd4
189537a6-5a83-44f5-8cdc-cf34aff42cc5,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,4809239c-fb16-41e8-ae07-8e4975eb9a1a
